In [22]:
import folium
import geopandas as gpd
import haversine as hs
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import osmnx as ox
import pandas as pd
import plotly_express as px
import plotly.io as pio
import plotly.offline as pyo

from branca.element import Figure
from folium import plugins
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from haversine import Unit
from shapely.geometry import Point, LineString
ox.config(use_cache=True, log_console=True)
pyo.init_notebook_mode(connected=True)

In [23]:
overall_data = pd.read_pickle("./2_all_graph_all_risk_added.pickle")
nodes_all, edges_all = ox.graph_to_gdfs(overall_data)

In [24]:
display(edges_all.head())
display(nodes_all.head())

osmid   lanes  \
u          v          key                                   
21569692   4682380491 0    [185913115, 401489878]  [5, 4]   
           6784129512 0                 401489883       6   
           6784129514 0                 401489877       3   
4682380491 21569692   0    [185913115, 401489878]  [5, 4]   
           4682381110 0                 474236841     NaN   

                                               name  highway  oneway   length  \
u          v          key                                                       
21569692   4682380491 0    Topanga Canyon Boulevard  primary   False  382.191   
           6784129512 0    Topanga Canyon Boulevard  primary   False   10.364   
           6784129514 0      Santa Susana Pass Road  primary   False   16.026   
4682380491 21569692   0    Topanga Canyon Boulevard  primary   False  382.191   
           4682381110 0                         NaN    track   False   40.470   

                             ref  \
u          v          key          
21569692   4682380491 0    CA 27   
           6784129512 0    CA 27   
           6784129514 0      NaN   
4682380491 21569692   0    CA 27   
           4682381110 0      NaN   

                                                                    geometry  \
u          v          key                                                      
21569692   4682380491 0    LINESTRING (-118.60483 34.27229, -118.60510 34...   
           6784129512 0    LINESTRING (-118.60483 34.27229, -118.60478 34...   
           6784129514 0    LINESTRING (-118.60483 34.27229, -118.60499 34...   
4682380491 21569692   0    LINESTRING (-118.60613 34.26908, -118.60611 34...   
           4682381110 0    LINESTRING (-118.60613 34.26908, -118.60590 34...   

                           distance_risk_score  max_speed_risk_score  \
u          v          key                                              
21569692   4682380491 0               0.888889                   0.4   
           6784129512 0               0.900000                   0.4   
           6784129514 0               0.900000                   0.4   
4682380491 21569692   0               0.888889                   0.4   
           4682381110 0               0.955556                   1.0   

                           density_risk_score  width_risk_score  \
u          v          key                                         
21569692   4682380491 0              0.414747               0.2   
           6784129512 0              0.000000               0.2   
           6784129514 0              0.000000               0.2   
4682380491 21569692   0              0.414747               0.2   
           4682381110 0              0.414747               1.0   

                           liq_risk_score  combined_risk      access bridge  \
u          v          key                                                     
21569692   4682380491 0               1.0       1.853635         NaN    NaN   
           6784129512 0               1.0       1.450000         NaN    NaN   
           6784129514 0               1.0       1.450000         NaN    NaN   
4682380491 21569692   0               1.0       1.853635         NaN    NaN   
           4682381110 0               1.0       2.620302  permissive    NaN   

                          maxspeed service tunnel width  
u          v          key                                
21569692   4682380491 0        NaN     NaN    NaN   NaN  
           6784129512 0        NaN     NaN    NaN   NaN  
           6784129514 0        NaN     NaN    NaN   NaN  
4682380491 21569692   0        NaN     NaN    NaN   NaN  
           4682381110 0        NaN     NaN    NaN   NaN

,y,x,highway,street_count,evacuation_flag,evacuation_name,ref,evacuation_type,geometry
osmid,,,,,,,,,
21569692,34.272295,-118.604826,traffic_signals,3,N,NA,NaN,NaN,POINT (-118.60483 34.27229)
4682380491,34.269075,-118.606130,NaN,3,N,NA,NaN,NaN,POINT (-118.60613 34.26908)
6784129512,34.272381,-118.604784,NaN,4,N,NA,NaN,NaN,POINT (-118.60478 34.27238)
6784129514,34.272341,-118.604991,NaN,4,N,NA,NaN,NaN,POINT (-118.60499 34.27234)
4682381110,34.269077,-118.605690,NaN,3,N,NA,NaN,NaN,POINT (-118.60569 34.26908)


In [27]:
fig=Figure(height=550,width=750)
m = folium.Map(location=[34.2203803, -118.5642751],
                     zoom_start = 12,
                     tiles='cartodbpositron')

fig.add_child(m)

In [40]:
evacuation_sites = nodes_all[nodes_all['evacuation_flag']=='Y']
evacuation_sites.columns = ['osmid', 'y', 'x', 'street_count', 'evacuation_flag', 'evacuation_name',
                           'ref', 'evacuation_type', 'geometry']
evacuation_sites.head()

,osmid,y,x,street_count,evacuation_flag,evacuation_name,ref,evacuation_type,geometry
osmid,,,,,,,,,
331380117,34.273145,-118.565802,NaN,3,Y,North Valley YMCA,NaN,shelter,POINT (-118.56580 34.27315)
4036602793,34.269342,-118.555279,NaN,3,Y,Limekiln Canyon Park,NaN,park,POINT (-118.55528 34.26934)
6711241701,34.234504,-118.487170,NaN,4,Y,Nordhoff Recreation Center,NaN,park,POINT (-118.48717 34.23450)
6984443265,34.240783,-118.504347,NaN,3,Y,Holmes Middle School,NaN,shelter,POINT (-118.50435 34.24078)
122598528,34.239477,-118.515618,NaN,4,Y,Northridge Academy High School Multipurpose Ro...,NaN,shelter,POINT (-118.51562 34.23948)


In [51]:
evacuation_sites.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 35 entries, 331380117 to 7552287091
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   osmid            35 non-null     float64 
 1   y                35 non-null     float64 
 2   x                2 non-null      object  
 3   street_count     35 non-null     int64   
 4   evacuation_flag  35 non-null     object  
 5   evacuation_name  35 non-null     object  
 6   ref              0 non-null      object  
 7   evacuation_type  35 non-null     object  
 8   geometry         35 non-null     geometry
dtypes: float64(2), geometry(1), int64(1), object(5)
memory usage: 2.7+ KB


In [61]:
for _evacuation_sites in evacuation_sites.iterrows():    
    if _evacuation_sites[1]['evacuation_type'] == 'shelter':
        lat = _evacuation_sites[1]['geometry'].y
        lon = _evacuation_sites[1]['geometry'].x
        
        
        folium.Marker([lat, lon], 
                  popup=_evacuation_sites[1]['evacuation_name'], 
                  tooltip=_evacuation_sites[1]['evacuation_name'], 
                  icon=folium.Icon(color="blue", icon="home", prefix='fa')
                 ).add_to(m)
    else:
        folium.Marker([lat, lon], 
                  popup=_evacuation_sites[1]['evacuation_name'], 
                  tooltip=_evacuation_sites[1]['evacuation_name'], 
                  icon=folium.Icon(color="green", icon="tree", prefix='fa')
                 ).add_to(m)
            
        
m

In [130]:
current_location = (34.2452, -118.4244)

In [131]:
def get_distance(loc1, loc2):
    result = hs.haversine(loc1,loc2, unit=Unit.METERS)
    return result

In [132]:
evacuation_sites['current_distance'] = \
evacuation_sites.apply(lambda x: get_distance(current_location, (x['geometry'].y, 
                                                                 x['geometry'].y)), axis=1)

/usr/local/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [133]:
edges_all.head()

osmid   lanes  \
u          v          key                                   
21569692   4682380491 0    [185913115, 401489878]  [5, 4]   
           6784129512 0                 401489883       6   
           6784129514 0                 401489877       3   
4682380491 21569692   0    [185913115, 401489878]  [5, 4]   
           4682381110 0                 474236841     NaN   

                                               name  highway  oneway   length  \
u          v          key                                                       
21569692   4682380491 0    Topanga Canyon Boulevard  primary   False  382.191   
           6784129512 0    Topanga Canyon Boulevard  primary   False   10.364   
           6784129514 0      Santa Susana Pass Road  primary   False   16.026   
4682380491 21569692   0    Topanga Canyon Boulevard  primary   False  382.191   
           4682381110 0                         NaN    track   False   40.470   

                             ref  \
u          v          key          
21569692   4682380491 0    CA 27   
           6784129512 0    CA 27   
           6784129514 0      NaN   
4682380491 21569692   0    CA 27   
           4682381110 0      NaN   

                                                                    geometry  \
u          v          key                                                      
21569692   4682380491 0    LINESTRING (-118.60483 34.27229, -118.60510 34...   
           6784129512 0    LINESTRING (-118.60483 34.27229, -118.60478 34...   
           6784129514 0    LINESTRING (-118.60483 34.27229, -118.60499 34...   
4682380491 21569692   0    LINESTRING (-118.60613 34.26908, -118.60611 34...   
           4682381110 0    LINESTRING (-118.60613 34.26908, -118.60590 34...   

                           distance_risk_score  max_speed_risk_score  \
u          v          key                                              
21569692   4682380491 0               0.888889                   0.4   
           6784129512 0               0.900000                   0.4   
           6784129514 0               0.900000                   0.4   
4682380491 21569692   0               0.888889                   0.4   
           4682381110 0               0.955556                   1.0   

                           density_risk_score  width_risk_score  \
u          v          key                                         
21569692   4682380491 0              0.414747               0.2   
           6784129512 0              0.000000               0.2   
           6784129514 0              0.000000               0.2   
4682380491 21569692   0              0.414747               0.2   
           4682381110 0              0.414747               1.0   

                           liq_risk_score  combined_risk      access bridge  \
u          v          key                                                     
21569692   4682380491 0               1.0       1.853635         NaN    NaN   
           6784129512 0               1.0       1.450000         NaN    NaN   
           6784129514 0               1.0       1.450000         NaN    NaN   
4682380491 21569692   0               1.0       1.853635         NaN    NaN   
           4682381110 0               1.0       2.620302  permissive    NaN   

                          maxspeed service tunnel width  
u          v          key                                
21569692   4682380491 0        NaN     NaN    NaN   NaN  
           6784129512 0        NaN     NaN    NaN   NaN  
           6784129514 0        NaN     NaN    NaN   NaN  
4682380491 21569692   0        NaN     NaN    NaN   NaN  
           4682381110 0        NaN     NaN    NaN   NaN

In [134]:
evacuation_sites.head()

,osmid,y,x,street_count,evacuation_flag,evacuation_name,ref,evacuation_type,geometry,current_distance
osmid,,,,,,,,,,
331380117,34.273145,-118.565802,NaN,3,Y,North Valley YMCA,NaN,shelter,POINT (-118.56580 34.27315),1.188277e+07
4036602793,34.269342,-118.555279,NaN,3,Y,Limekiln Canyon Park,NaN,park,POINT (-118.55528 34.26934),1.188302e+07
6711241701,34.234504,-118.487170,NaN,4,Y,Nordhoff Recreation Center,NaN,park,POINT (-118.48717 34.23450),1.188531e+07
6984443265,34.240783,-118.504347,NaN,3,Y,Holmes Middle School,NaN,shelter,POINT (-118.50435 34.24078),1.188490e+07
122598528,34.239477,-118.515618,NaN,4,Y,Northridge Academy High School Multipurpose Ro...,NaN,shelter,POINT (-118.51562 34.23948),1.188498e+07


In [166]:
merge_information = evacuation_sites.merge(edges_all.reset_index(drop=True), how='inner', on='ref')
merge_information.shape

(1690640, 29)

In [167]:
merge_information = merge_information.drop_duplicates(subset='evacuation_name', keep="last")
merge_information.shape

(35, 29)

In [168]:
merge_information

,osmid_x,y,x,street_count,evacuation_flag,evacuation_name,ref,evacuation_type,geometry_x,current_distance,...,density_risk_score,width_risk_score,liq_risk_score,combined_risk,access,bridge,maxspeed,service,tunnel,width
48303,34.273145,-118.565802,NaN,3,Y,North Valley YMCA,NaN,shelter,POINT (-118.56580 34.27315),1.188277e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
96607,34.269342,-118.555279,NaN,3,Y,Limekiln Canyon Park,NaN,park,POINT (-118.55528 34.26934),1.188302e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
144911,34.234504,-118.487170,NaN,4,Y,Nordhoff Recreation Center,NaN,park,POINT (-118.48717 34.23450),1.188531e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
193215,34.240783,-118.504347,NaN,3,Y,Holmes Middle School,NaN,shelter,POINT (-118.50435 34.24078),1.188490e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
241519,34.239477,-118.515618,NaN,4,Y,Northridge Academy High School Multipurpose Ro...,NaN,shelter,POINT (-118.51562 34.23948),1.188498e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
289823,34.260862,-118.579834,NaN,4,Y,Mason Park Recreation Center,NaN,shelter,POINT (-118.57983 34.26086),1.188358e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
338127,34.260849,-118.579549,NaN,3,Y,Mason Park,NaN,park,POINT (-118.57955 34.26085),1.188358e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
386431,34.242784,-118.466380,NaN,3,Y,Sepulveda Middle School Gym,NaN,shelter,POINT (-118.46638 34.24278),1.188477e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
434735,34.242740,-118.592048,NaN,3,Y,William Tell Aggeler Opportunity High School,NaN,shelter,POINT (-118.59205 34.24274),1.188477e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
483039,34.250855,-118.553747,NaN,3,Y,Nobel Middle School,NaN,shelter,POINT (-118.55375 34.25085),1.188424e+07,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
top_10 = merge_information.sort_values('current_distance', ascending=False).head(5)
top_10.shape

(5, 29)

### Using dijkstra

In [175]:
fig=Figure(height=550,width=750)
m2 = folium.Map(location=[34.2203803, -118.5642751],
                     zoom_start = 12,
                     tiles='cartodbpositron')

fig.add_child(m2)

In [177]:
folium.Marker(current_location,
              icon=folium.plugins.BeautifyIcon(border_color='Darkgreen',
                           text_color='darkgreen',
                           number=0,
                           icon_shape='marker'),
  tooltip='Your Current Location'
 ).add_to(m2)

ctr = 0
for _top_10 in top_10.iterrows():
    print(_top_10[1]['evacuation_name'])
    print(_top_10[1]['combined_risk'])
    ctr +=1
    orig = ox.get_nearest_node(overall_data, current_location)
    dest = ox.get_nearest_node(overall_data, (_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x))
    route = nx.shortest_path(overall_data, orig, dest, weight='combined_risk', method='dijkstra')
    
    coords_1 = [] 
    coords_1.append(current_location)
    for i in route:
        point = overall_data.nodes[i]
        coords_1.append([point['y'], point['x']])

    coords_1.append([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x])


    if ctr == 1:
        folium.Marker([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x],
                      icon=folium.plugins.BeautifyIcon(border_color='Red',
                                   text_color='red',
                                   number=ctr,
                                   icon_shape='marker'),
                        tooltip=_top_10[1]['evacuation_name']
                     ).add_to(m2)

        folium.PolyLine(coords_1, popup='<b>Path of Vehicle_1</b>',
                                 tooltip='Vehicle_1',
                                 color='red',
                                 weight=5).add_to(m2)

    else:
        folium.Marker([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x],
                      icon=folium.plugins.BeautifyIcon(border_color='Blue',
                                   text_color='blue',
                                   number=ctr,
                                   icon_shape='marker'),
                      tooltip=_top_10[1]['evacuation_name']
                     ).add_to(m2)

        folium.PolyLine(coords_1, popup='<b>Path of Vehicle_1</b>',
                                 tooltip='Vehicle_1',
                                 color='blue',
                                 weight=3).add_to(m2)
        
m2

Northridge Middle School (Multipurpose Room) 2 of 2
0.0


/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.



Northridge Middle School (Gym) 1 of 2
0.0
North Hills Community Park
0.0


/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.



Wilkinson Multipurpose Senior Citizen Center
0.0


/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.



Vanalden Park
0.0


/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.



### Using A Star

In [178]:
fig=Figure(height=550,width=750)
m3 = folium.Map(location=[34.2203803, -118.5642751],
                     zoom_start = 12,
                     tiles='cartodbpositron')

fig.add_child(m3)

In [180]:
folium.Marker(current_location,
              icon=folium.plugins.BeautifyIcon(border_color='Darkgreen',
                           text_color='darkgreen',
                           number=0,
                           icon_shape='marker'),
  tooltip='Your Current Location'
 ).add_to(m3)

ctr = 0
for _top_10 in top_10.iterrows():
    ctr +=1
    orig = ox.get_nearest_node(overall_data, current_location)
    dest = ox.get_nearest_node(overall_data, (_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x))
    #route = nx.shortest_path(overall_data, orig, dest, weight='combined_risk', method='dijkstra')
    route = nx.astar_path(overall_data, orig, dest, weight='combined_risk', heuristic=None)
    
    coords_1 = [] 
    coords_1.append(current_location)
    for i in route:
        point = overall_data.nodes[i]
        coords_1.append([point['y'], point['x']])

    coords_1.append([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x])


    if ctr == 1:
        folium.Marker([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x],
                      icon=folium.plugins.BeautifyIcon(border_color='Red',
                                   text_color='red',
                                   number=ctr,
                                   icon_shape='marker'),
                        tooltip=_top_10[1]['evacuation_name']
                     ).add_to(m3)

        folium.PolyLine(coords_1, popup='<b>Path of Vehicle_1</b>',
                                 tooltip='Vehicle_1',
                                 color='red',
                                 weight=5).add_to(m3)

    else:
        folium.Marker([_top_10[1]['geometry_x'].y, _top_10[1]['geometry_x'].x],
                      icon=folium.plugins.BeautifyIcon(border_color='Blue',
                                   text_color='blue',
                                   number=ctr,
                                   icon_shape='marker'),
                      tooltip=_top_10[1]['evacuation_name']
                     ).add_to(m3)

        folium.PolyLine(coords_1, popup='<b>Path of Vehicle_1</b>',
                                 tooltip='Vehicle_1',
                                 color='blue',
                                 weight=3).add_to(m3)
        
m3

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.

/usr/local/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning:

The `get_nearest